# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

We want to be able to train our model on a hardware accelerator like the GPU or MPS, if available. Let’s check to see if torch.cuda or torch.backends.mps are available, otherwise we use the CPU.

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# Define the Class

We define our neural network by subclassing ```nn.Module```, and initialize the neural network layers in ```__init__```. Every nn.Module subclass implements the operations on input data in the ```forward``` method.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ```NeuralNetwork```, and move it to the ```device```, and print its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s ```forward```, along with some background operations. Do not call ```model.forward()``` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the ```nn.Softmax``` module.

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5])


# Model Layers
Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


# nn.Flatten
We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


# nn.Linear
The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


# nn.ReLU
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2218,  0.0325,  0.4958, -0.1925,  0.1113,  0.3551,  0.3102, -0.2189,
          0.0169, -0.1373,  0.0349,  0.4839, -0.3914,  0.2398, -0.0118,  0.7987,
          0.2542, -0.1232,  0.1633,  0.0541],
        [-0.1995,  0.0642,  0.1889, -0.2877,  0.0985,  0.1111, -0.0270, -0.1836,
         -0.1865, -0.0481,  0.1834,  0.5445, -0.1514,  0.2241,  0.1250,  0.4858,
         -0.0015, -0.2512,  0.2680, -0.0135],
        [ 0.0643,  0.4016,  0.5214, -0.3281,  0.0877,  0.1097,  0.2990, -0.0026,
         -0.2741, -0.0614,  0.2415,  0.5083, -0.4713,  0.2356,  0.1423,  0.5912,
          0.1044, -0.0606,  0.1935, -0.2749]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0325, 0.4958, 0.0000, 0.1113, 0.3551, 0.3102, 0.0000, 0.0169,
         0.0000, 0.0349, 0.4839, 0.0000, 0.2398, 0.0000, 0.7987, 0.2542, 0.0000,
         0.1633, 0.0541],
        [0.0000, 0.0642, 0.1889, 0.0000, 0.0985, 0.1111, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1834, 0.5445, 0.0000, 0.2241, 0.12

# nn.Sequential
nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

# nn.Softmax
The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Model Parameters
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. 
Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s ```parameters()``` or ```named_parameters()``` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 2.4071e-02,  2.7820e-02,  1.1799e-02,  ..., -2.3759e-02,
          3.2961e-02, -9.0335e-05],
        [-3.4845e-02, -2.2521e-03,  2.3511e-02,  ..., -1.4420e-02,
          2.0285e-02,  1.4066e-02]], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0217, -0.0016], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0205, -0.0027,  0.0430,  ..., -0.0067, -0.0066, -0.0339],
        [-0.0340,  0.0107,  0.0053,  ..., -0.0242, -0.0139, -0.0158]],
       gra